<a href="https://colab.research.google.com/github/Sambosis/NowNotThen/blob/circleci-project-setup/Capacity_VRP_ORTools_Detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ortools pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Capacity Vehicle Routing Problem using Google OR-Tools

This notebook presents a solution to the Capacity Vehicle Routing Problem (CVRP) using Google OR-Tools. The CVRP is a classic optimization problem in logistics where the goal is to determine the optimal set of routes for a fleet of vehicles delivering to a set of customers, subject to constraints on vehicle capacity and route length.

The solution is divided into several sections, each performing a specific part of the problem-solving process:

1. **Data Loading and Preparation:** In this section, we load the necessary data from CSV files and prepare it for use in the optimization model. This includes creating a distance matrix and a list of customer demands.

2. **Model Creation:** In this section, we create the optimization model, define the objective function and add the necessary constraints.

3. **Problem Solving:** In this section, we solve the optimization problem and obtain the optimal set of routes.

4. **Solution Presentation:** In this section, we present the solution in a human-readable format, showing the routes for each vehicle and the total distance and demand served.

5. **Visualization:** In this section, we visualize the routes on a map, providing a graphical representation of the solution.

# 1. Data Loading and Preparation

In this section, we load the necessary data from CSV files and prepare it for use in the optimization model. This includes creating a distance matrix and a list of customer demands.

We load two CSV files:

- `distances.csv`: This file contains the distances between customers. The first row and column are customer numbers and their intersection in the grid is their distance apart. The 0,0 location of the CSV is blank.

- `dataset.csv`: This file contains information about the customers. The customer numbers are in the 'CUST NUMB' column, the name of the customer is in the 'CUSTOMER NAME' column, and the demand of each stop is in the 'Companion' column. There are also columns for 'Latitude' and 'Longitude' for visualizing the stops on the map.

We create a distance matrix as a numpy array from the distances in `distances.csv` and a dictionary mapping customer numbers to their demands from the 'Companion' column in `dataset.csv`.

In [5]:
# Import necessary libraries
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

# Load your data
distances_df = pd.read_csv('distances.csv', index_col=0)
dataset = pd.read_csv('dataset.csv')

# Prepare your data
# Convert the distances dataframe to a numpy array to create a distance matrix
distance_matrix = distances_df.values

# Create a dictionary mapping customer numbers to their demands
# Create a dictionary mapping customer numbers to their demands
capacities = dataset.set_index('CUST NUMB')['Companion'].to_dict()
capacities = {int(k): v for k, v in capacities.items()}
#print(capacities)
#capacities = dataset.set_index('CUST NUMB')['Companion'].to_dict()

# Create a dictionary mapping customer numbers to their names for later use
customer_names = dataset.set_index('CUST NUMB')['CUST NAME'].to_dict()
#print(customer_names)
# Create a dictionary mapping customer numbers to their coordinates for visualization
coordinates = dataset.set_index('CUST NUMB')[['Latitude', 'Longitude']].to_dict('index')
print(coordinates)

{650476: {'Latitude': 39.498835375174224, 'Longitude': -76.65440171080951}, 6501024: {'Latitude': 39.49678997314139, 'Longitude': -76.65442382698039}, 65050465: {'Latitude': 39.49423458137511, 'Longitude': -76.6488292951814}, 226500031: {'Latitude': 39.49574478353152, 'Longitude': -76.65591867386789}, 176500031: {'Latitude': 39.26864052824725, 'Longitude': -76.61164370402383}, 196500032: {'Latitude': 39.27317480164636, 'Longitude': -76.60541519442452}, 226500030: {'Latitude': 39.28050390587434, 'Longitude': -76.60613127707688}, 650707: {'Latitude': 39.26987341883082, 'Longitude': -76.60544991566691}, 156500184: {'Latitude': 39.27608680234434, 'Longitude': -76.60445319828649}, 226500027: {'Latitude': 39.497761709561935, 'Longitude': -76.6518986873857}, 176500024: {'Latitude': 39.268595092694106, 'Longitude': -76.59776391602836}, 650643: {'Latitude': 39.27325558869548, 'Longitude': -76.50237091220987}, 650938: {'Latitude': 39.28612949198583, 'Longitude': -76.60203142027805}, 196500020: {

# 2. Model Creation

In this section, we create the optimization model, define the objective function and add the necessary constraints.

We use Google OR-Tools to create a routing model. The routing model is a powerful tool for solving complex routing problems. It provides a high-level interface for defining the problem and uses efficient algorithms to solve it.

We define the objective function as the total distance traveled by all vehicles. The model tries to minimize this total distance.

We add two types of constraints to the model:

- **Route constraints:** These constraints ensure that each customer is visited exactly once by exactly one vehicle.

- **Capacity constraints:** These constraints ensure that the total demand served by each vehicle does not exceed its capacity. The capacities of the vehicles are defined as [7000, 15000, 12500].

In [6]:
# Define vehicle capacities
vehicle_capacities = [7000, 15000, 12500]

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), len(vehicle_capacities), 0)
print
# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback
def distance_callback(from_index, to_index):
    return distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint
def demand_callback(from_index):
    return capacities[manager.IndexToNode(from_index)]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    vehicle_capacities,  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

True

# 3. Problem Solving

In this section, we solve the optimization problem and obtain the optimal set of routes.

We define the search parameters for the solver, including the search strategy and the time limit. We use the guided local search metaheuristic, which is a good choice for vehicle routing problems.

We then call the `SolveWithParameters` method of the routing model to solve the problem. This method returns a solution, which is an object that contains the optimal set of routes.

In [7]:
# Set search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 30
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

# 4. Solution Presentation

In this section, we present the solution in a human-readable format, showing the routes for each vehicle and the total distance and demand served.

We iterate over each vehicle and each node in its route, printing the customer number and name, the demand served, and the cumulative distance traveled. We also keep track of the total distance and demand served by all vehicles.

Finally, we print the total distance and demand served by all vehicles.

In [8]:
# Print solution
total_distance = 0
total_load = 0
for vehicle_id in range(len(vehicle_capacities)):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
    route_distance = 0
    route_load = 0
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        next_node_index = manager.IndexToNode(solution.Value(routing.NextVar(index)))
        route_load += capacities.get(node_index, 0)
        route_distance += routing.GetArcCostForVehicle(node_index, next_node_index, vehicle_id)
        plan_output += ' {} Load({}) -> '.format(customer_names.get(node_index, 'Unknown'), route_load)
        index = solution.Value(routing.NextVar(index))
    node_index = manager.IndexToNode(index)
    total_distance += route_distance
    total_load += route_load
    plan_output += ' {} Load({})\n'.format(customer_names.get(node_index, 'Unknown'), route_load)
    plan_output += 'Distance of the route: {}m\n'.format(route_distance)
    plan_output += 'Load of the route: {}\n'.format(route_load)
    print(plan_output)
print('Total distance of all routes: {}m'.format(total_distance))
print('Total load of all routes: {}'.format(total_load))

Route for vehicle 0:
 Unknown Load(0) ->  Unknown Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 Unknown Load(0) ->  Unknown Load(0)
Distance of the route: 0m
Load of the route: 0



SystemError: ignored

# 5. Visualization

In this section, we visualize the routes on a map, providing a graphical representation of the solution.

We use the `folium` library to create a map and add markers for each customer. We then add lines between the customers to represent the routes. Each route is color-coded by vehicle.

Finally, we display the map.

In [ ]:
import folium
from folium import plugins

# Create a map centered around the coordinates of the first customer
m = folium.Map(location=[coordinates[0]['Latitude'], coordinates[0]['Longitude']], zoom_start=12)

# Add markers for each customer
for node_index in range(len(distance_matrix)):
    folium.Marker([coordinates[node_index]['Latitude'], coordinates[node_index]['Longitude']],
                  popup='{}: {}'.format(node_index, customer_names[node_index]),
                  icon=folium.Icon(color='blue')).add_to(m)

# Add lines representing the routes
colors = ['red', 'green', 'blue']
for vehicle_id in range(len(vehicle_capacities)):
    route_coordinates = []
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route_coordinates.append([coordinates[node_index]['Latitude'], coordinates[node_index]['Longitude']])
        index = routing.NextVar(index).solution_value()
    node_index = manager.IndexToNode(index)
    route_coordinates.append([coordinates[node_index]['Latitude'], coordinates[node_index]['Longitude']])
    folium.PolyLine(route_coordinates, color=colors[vehicle_id % len(colors)], weight=2.5, opacity=1).add_to(m)

# Display the map
m